In [9]:
from langchain_community.document_loaders import PyPDFLoader
 
def extract_text_from_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    texts = ""
    for doc in documents:
        texts += doc.page_content + " "
    return texts

docs = extract_text_from_pdf("./data/Uploads/Mutalib_2019_IOP_Conf._Ser.__Mater._Sci._Eng._469_012092.pdf")
print(docs)

IOP Conference Series:
Materials Science and
Engineering
      
PAPER • OPEN ACCESS
Magnetorheological finishing on metal surface: A
review
To cite this article: N A Mutalib et al 2019 IOP Conf. Ser.: Mater. Sci. Eng. 469 012092
 
View the article online for updates and enhancements.
You may also like
Synthesis of Magnetorheological Fluid
using Electrolytic Carbonyl Iron Powder for
Enhanced Stability
Rakesh Erappa Jinaga and T Jagadeesha
-
Development of Vibration Isolator
Magnetorheological Elastomer Based
G Priyandoko, P Suwandono, NR Ismail et
al.
-
Magnetorheological torque transmission
devices with permanent magnets
H Böse, T Gerlach and J Ehrlich
-
 
This content was downloaded from IP address 65.1.45.255 on 25/02/2025 at 08:39 Content from this work may be used under the terms of the Creative Commons Attribution 3.0 licence. Any further distribution
of this work must maintain attribution to the author(s) and the title of the work, journal citation and DOI.
Published under licenc

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size=200, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    return text_splitter.split_text(documents)

chunks = split_docs(docs)
print(len(chunks))
print('First Chunk\n', chunks[0])
print('\n\n\nsecond chunk\n', chunks[1])

231
First Chunk
 IOP Conference Series:
Materials Science and
Engineering
      
PAPER • OPEN ACCESS
Magnetorheological finishing on metal surface: A
review



second chunk
 review
To cite this article: N A Mutalib et al 2019 IOP Conf. Ser.: Mater. Sci. Eng. 469 012092
 
View the article online for updates and enhancements.
You may also like


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from uuid import uuid4

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

index = faiss.IndexFlatL2(len(embedding_model.embed_query(chunks[0])))

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
documents = [Document(page_content=chunk, metadata={'chunk_id':i}) for i, chunk in enumerate(chunks)]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)


['712f90f4-a385-4af3-96e1-19a0eb78bacd',
 'ef964e5a-525b-474c-9ab4-8b0601d4953b',
 'a3b1c3d8-efc3-4e4d-950d-67672fdc45c7',
 '5ea08935-44ec-402c-bd98-3e8fcb82a846',
 '65995f08-b127-43af-a103-ece9038a11b4',
 'fc3a4ac5-81b5-481b-bea0-386cea5cd9ae',
 'bf138152-f1b8-480c-af13-128a7b0ae495',
 '4ecb142c-6b72-4c18-947a-db46efe01011',
 '8990fadc-69b1-45ba-8f20-6cf15e43683b',
 '87eb32b1-615b-42de-8398-976f1179d7e5',
 'bd3259f9-ce99-4fec-8c3e-057364e37d58',
 '380b2d69-b158-4191-b398-5793dd9ff634',
 '65e4f2a8-61b2-49e4-8a63-c8750ff62414',
 'f988b178-b517-4dd9-8716-ced89c049fbc',
 '8a075efe-9655-4b8f-9021-6f6b2a178138',
 '0234555c-d7fa-4440-b731-43a291ead100',
 '16cd07c2-d451-4f42-a666-59dcfb343c97',
 'ebc1ac52-f6cd-4d09-b4d8-158645e44c8a',
 '63db4201-14f0-424e-9172-b7bfd1fc43de',
 '15b4dc06-57c0-4efa-942b-e53ff110759a',
 '6816f051-f3ad-4823-8a23-d056b1301f34',
 '5246ae2e-1074-4dbf-a79b-3855591960dd',
 'f5524398-bbfa-412e-a6ff-bf230d90d578',
 'cc706889-7f7a-4bba-8155-4b84fd510b9c',
 '80ed4c33-b7c4-

In [12]:
embedding_vector = index.reconstruct(0)
print("Embedding vector shape:", embedding_vector.shape)
print("Embedding vector:", embedding_vector)

Embedding vector shape: (384,)
Embedding vector: [-4.27920334e-02  4.62989882e-03  3.73352803e-02  1.81439817e-02
  4.31342423e-02 -2.59420443e-02 -2.76514404e-02 -3.69591452e-02
 -9.31640118e-02 -6.06667809e-02 -1.08544990e-01  5.49387336e-02
  6.07107207e-03  1.05600934e-02 -6.31525964e-02  2.01729257e-02
  2.44991301e-04 -6.89911768e-02  6.06481992e-02  4.08921801e-02
  3.88053134e-02  1.89966783e-02  4.16894369e-02 -7.15586618e-02
 -3.78946513e-02  5.10967113e-02  1.02987923e-02 -3.17859426e-02
  1.42898019e-02 -2.15978306e-02  5.46289235e-02 -8.45311489e-03
 -1.31047377e-02 -8.11325610e-02  4.52518724e-02 -2.03848276e-02
 -9.53565463e-02  4.70087714e-02 -2.27881614e-02 -2.56955959e-02
 -3.49350534e-02 -6.47842437e-02 -4.00958629e-03 -3.07174381e-02
  1.03715152e-01  2.29896009e-02  1.04387552e-01 -8.56211483e-02
  6.20598299e-03  2.46601701e-02 -3.22496556e-02 -1.71742644e-02
  4.14946750e-02 -9.19486508e-02  2.58763656e-02  7.56571963e-02
 -1.60024092e-02 -1.10764749e-01 -1.01333

In [13]:
query = "manufacturing"  # or any topic/question
results = vector_store.similarity_search(query, k=3)
for doc in results:
    print(doc.page_content)

finishing (MRAFF) process . International Journal of Machine Tools & Manufacture . 44: p. 
1019–1029.
1Faculty of Manufacturing Engineering, University Malaysia Pahang, 
26600 Pekan, Pahang, Malaysia 
2Faculty of Mechanical Engineering, University Malaysia Pahang, 26600 Pekan, 
Pahang, Malaysia
silicon by magnetic field -assisted finishing process . International Journal of Advanced 
Manufacturing Technology. 55: p. 243-252.


In [14]:
from langchain_huggingface import HuggingFaceEndpoint
from dotenv import load_dotenv

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="google/flan-t5-large",
    task="text-generation",
    max_new_tokens=256,
    temperature=0.3,
)

context = " ".join([doc.page_content for doc in results])

prompt = f"Generate a multiple-choice question based on the following text:\n\n{context}\n\nProvide 4 options and indicate the correct answer."

question = llm.invoke(prompt)
print(question)

StopIteration: 